In [1]:
import tesstest
import tesstrain
from os import listdir
from os.path import join, isfile

In [3]:
# see ./Dockerfile (scribble_train base image) for more information on the file structure inside this container
TEST_PATH = '/train/tesstrain/data/storysquad-ground-truth' 
model_name = "kaggle"
tessdata="/train/tessdata"

In [4]:
# train a tesseract model for OCR
# see ./structured_experiments/2022.04.28.notebook_template/mytesseract/tesstrain/__init__.py for implementation
tesstrain.train_tesseract("/train/tesstrain",  model_name=model_name, start_model="eng", tessdata=tessdata, max_iterations=100)

Running C code (via make) to train a tesseract model: cd /train/tesstrain && make training TESSDATA_FOLDER=/train/tessdata MODEL_NAME=kaggle START_MODEL=eng TESSDATA=/train/tessdata MAX_ITERATIONS=100
message: Command 'cd /train/tesstrain && make training TESSDATA_FOLDER=/train/tessdata MODEL_NAME=kaggle START_MODEL=eng TESSDATA=/train/tessdata MAX_ITERATIONS=100' returned non-zero exit status 2.


make: *** [Makefile:185: data/kaggle/list.train] Error 1


In [4]:
# get a random sample of images from TEST_PATH folder and read their ground truth files (labels with correct OCR)
test_image_paths = []
test_image_labels = []
count = 0

for file_name in listdir(TEST_PATH):
    file_path = join(TEST_PATH, file_name)
    if count < 100 and file_path.endswith(".png") and isfile(file_path):
        
        ground_truth_path = file_path.replace(".png", ".gt.txt")
        if isfile(file_path) and isfile(ground_truth_path):
            test_image_paths.append(file_path)
            with open(ground_truth_path, mode='r') as f:
                ground_truth = f.read()
            test_image_labels.append(ground_truth)
            count += 1
    else:
        continue
print(test_image_paths[:2], test_image_labels[:2])

['/train/tesstrain/data/storysquad-ground-truth/31-3117-1-1.png', '/train/tesstrain/data/storysquad-ground-truth/31-3112-1-14-1.png'] ['gum drop land', 'The end']


In [5]:
# generate word error rate and character error rate for model based on random sample generated above
# see ./structured_experiments/2022.04.28.notebook_template/mytesseract/tesstrain/__init__.py for implementation
word_error_rate, char_error_rate = tesstest.test_model("kaggle", tessdata, test_image_paths, test_image_labels)

In [6]:
print(f"Word Error Rate of {round(100 *word_error_rate, 2)}% and Character Error Rate of {round(100 *char_error_rate, 2)}%")

Word Error Rate of 100.0% and Character Error Rate of 92.87%
